(This notebook is for **the week 3** assignment of the **Coursera Applied Data Science Capstone** module.)
### Segmenting and Clustering Neighborhoods in Toronto

In [52]:
# install web-scraper (following: https://youtu.be/ng2o98k983k)
#! pip install beautifulsoup4 # uncomment if not installed
#! pip install lxml # uncomment if not installed
#! pip intsall html5lib # uncomment if not installed
#! pip install requests # uncomment if not installed

Import libraries and get wikipedia page content.

In [53]:
import pandas as pd # import pandas library
import requests # import requests library
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # url to be scraped
page_response = requests.get(url) # fetch the url content using the requests library
src=page_response.content # set the content of the repsonse to a variable #https://realpython.com/python-requests/

Import the beautiful soup library, create the soup object and output the page title.

In [54]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(src, 'lxml') # pass the variable to BeautifulSoup, https://youtu.be/87Gx3U0BDlo                    
#print(soup)
soup.title.text # output the title of the soup object #https://www.pluralsight.com/guides/extracting-data-html-beautifulsoup

'List of postal codes of Canada: M - Wikipedia'

Create a variable for the table only. Find all the tr tagged data in the table.

In [55]:
table=soup.find("table",{"class":"wikitable sortable"})  #https://stackoverflow.com/questions/33766740/beautifulsoup-find-table-with-specified-class-on-wikipedia-page
table_data = table.tbody.find_all("tr") # in the tabe body find all tr tags 
# uncomment a line to check the data
#print(table) 
#df.head() 
#print(table.title) 
#print(table_data) 

Get and print the table headings.

In [56]:
headings = []
for th in table_data[0].find_all("th"):
    headings.append(th.text.replace('\n',''))
print(headings) # list of table headings

['Postcode', 'Borough', 'Neighborhood']


Get the data from the table for one row to check it.

In [57]:
data_list = []
for td in table_data[287].find_all("td"):
    data_list.append(td.text.replace('\n',''))
print(data_list)

['M9Z', 'Not assigned', 'Not assigned']


Get the data from the table for **all rows**.

In [58]:
data_list = [] # variable ofr list off all data we want
for i in range(0,len(table_data)-1):
    for td in table_data[i].find_all("td"): # finds all the td tags in the html
        data_list.append(td.text.replace('\n','')) # add the row to the list
#print(data_list) # uncomment to check

Sanity check the result.

In [59]:
print('Length of list = ', len(data_list), sep='')
print('Number of columns = ', len(headings), sep='')
print('Number of rows in the table = ', int(len(data_list)/len(headings)), sep='')

Length of list = 858
Number of columns = 3
Number of rows in the table = 286


Reshape the list as an array and cast to a dataframe.

In [60]:
import numpy as np
reshaped_array=np.reshape(np.asarray(data_list), (int(len(data_list)/len(headings)) , len(headings))) # reshape the array
df=pd.DataFrame(reshaped_array) # make the result a dataframe
df.columns=headings # assign the table headings as the column names
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Condition to find the 'Not Assigned' **Boroughs**, and remove them.

In [61]:
condition=df['Borough']=='Not assigned' # the condition that we don't want https://github.com/pandas-dev/pandas/issues/20944
df=df[~condition] # pick rows that do not (~) satisfy the condition
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Group the dataframe by **'Postcode'** and **'Borough'** and apply the join function to the grouped **'Neighborhood'** data.

In [62]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join) #https://realpython.com/pandas-groupby/
df=df.to_frame().reset_index() # reset index
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Condition to find the 'Not Assigned' **Neighborhoods**, and remove them.

In [64]:
condition=df['Neighborhood']=='Not assigned' # the condition that we don't want https://github.com/pandas-dev/pandas/issues/20944
for i in range(0,len(condition-1)):
    if condition[i]==True:
        df['Neighborhood'][i]=df['Borough'][i]      
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [65]:
print('Number of rows in the dataframe = ', df.shape[0], sep='')

Number of rows in the dataframe = 103
